<a href="https://colab.research.google.com/github/FULLSTACK-11/RPA_ECOM/blob/main/RPA_ECOM_Cron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pdfs_module_integrated.py

# =========================================
# 1) INSTALAR DEPENDENCIAS (en Colab)
# =========================================
!apt-get update && apt install -y chromium-chromedriver
!pip install selenium pandas xlrd requests PyPDF2
!pip install xlrd==1.2.0


import os
import sys
import glob
import time
import base64
import json
import re
from datetime import datetime, timedelta

import requests
from PyPDF2 import PdfReader, PdfWriter
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# -----------------------------------------
# 2) INPUT USUARIO
# -----------------------------------------
planilla_id ="47527"
idDriver    =108

# -----------------------------------------
# 3) CONFIGURACIONES GENERALES
# -----------------------------------------
download_path   = "/content/Compilados"
conversion_path = os.path.join(download_path, "conversion")
pdfs_path       = os.path.join(download_path, "pdfs")
for p in (download_path, conversion_path, pdfs_path):
    os.makedirs(p, exist_ok=True)

usuario           = "it@servimax"
contraseña        = "Servimax2025&"
convertapi_secret = "secret_2vaBCd8hKKwyMg7o"
driver_mapping    = {
    109:[10510,10513],105:[10518,10517],101:[10511,10512],
    104:[10507,10509],120:[10503,10516],106:[10514,10520],
    108:[10508,10504],121:[10501,10506],115:[10515,10519],
    103:[10502,10505]
}

if idDriver in driver_mapping:
    asesor1, asesor2 = map(str, driver_mapping[idDriver])
    print(f"✅ Conductor {idDriver} → Asesores: {asesor1}, {asesor2}")
else:
    asesor1, asesor2 = "10503","10516"
    print(f"⚠️ ID {idDriver} no mapeado, uso por defecto {asesor1},{asesor2}")

# -----------------------------------------
# 4) SELENIUM SETUP
# -----------------------------------------
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_path}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# -----------------------------------------
# 5) FUNCIONES AUXILIARES GENERALES
# -----------------------------------------
def esperar_archivo_descargado(nombre, timeout=30):
    start = time.time()
    while True:
        ruta = os.path.join(download_path, nombre)
        if os.path.exists(ruta):
            return ruta
        if time.time() - start > timeout:
            raise TimeoutException(f"⏳ Tiempo agotado esperando {nombre}")
        time.sleep(0.5)

def get_fecha_pedidos():
    hoy = datetime.today()
    dias = 2 if hoy.weekday() == 0 else 1
    return (hoy - timedelta(days=dias)).strftime("%Y-%m-%d")

# -----------------------------------------
# 6) DESCARGA XLS (Pedidos, Facturas, Entregas)
# -----------------------------------------
def descargar_pedidos(driver, asesor_id, prefijo):
    try:
        driver.get("https://myeforce.ecom.com.co//eforce/modulos/pedidosNewpru2/index.php")
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.NAME,"ped_usuari")))
        Select(driver.find_element(By.NAME,"ped_usuari")).select_by_value(asesor_id)
        driver.execute_script("document.getElementById('b_fecha1').value=arguments[0];", get_fecha_pedidos())
        driver.execute_script("document.getElementsByName('ped_estado')[0].value='1';")
        driver.execute_script("document.getElementsByName('limit')[0].value='1000';")
        driver.find_element(By.ID,"read").click()
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"{prefijo}_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ pedidos {prefijo}: {e}")

def descargar_facturas(driver, asesor_id, prefijo):
    try:
        driver.get("https://myeforce.ecom.com.co//eforce/modulos/informes/vtafacturas/index.php")
        Select(driver.find_element(By.ID,"select_0")).select_by_value("10059830")
        time.sleep(1)
        driver.execute_script(
            "document.getElementsByName('fecha1')[0].value=arguments[0];",
            datetime.today().strftime("%Y-%m-%d")
        )
        Select(driver.find_element(By.ID,"select_1")).select_by_value(asesor_id)
        driver.find_element(By.XPATH,"//input[@value='Generar Informe']").click()
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"{prefijo}_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ facturas {prefijo}: {e}")

def descargar_entregas(driver):
    try:
        url = f"https://myeforce.ecom.com.co//eforce/modulos/entrega/index.php?modo=detalles&pla_numero={planilla_id}"
        driver.get(url); time.sleep(2)
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"Entregas_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ entregas: {e}")

# -----------------------------------------
# 7) CONVERTIR XLS -> CSV VIA ConvertAPI
# -----------------------------------------
def convertir_a_csv_convertapi():
    url = "https://v2.convertapi.com/convert/xls/to/csv"
    headers = {"Authorization": f"Bearer {convertapi_secret}"}
    archivos = glob.glob(os.path.join(download_path,"*.xls"))
    if not archivos:
        print("⚠️ No hay .xls para convertir")
        return
    print("\n🔄 ConvertAPI XLS -> CSV")
    for xls in archivos:
        name = os.path.basename(xls)
        csv_name = name.replace(".xls",".csv")
        csv_path = os.path.join(conversion_path, csv_name)
        raw = open(xls, "rb").read()
        b64 = base64.b64encode(raw).decode()
        payload = {
            "Parameters": [
                {"Name":"File","FileValue":{"Name":name,"Data":b64}},
                {"Name":"StoreFile","Value":True}
            ]
        }
        r = requests.post(url, headers=headers, json=payload)
        if r.ok:
            csv_url = r.json()["Files"][0]["Url"]
            r2 = requests.get(csv_url)
            if r2.ok:
                open(csv_path,"wb").write(r2.content)
                print(f"✅ {csv_name}")
            else:
                print(f"❌ error descargar CSV {csv_name}")
        else:
            print(f"❌ ConvertAPI {name}: {r.text}")

# -----------------------------------------
# 8) DESCARGA DE PDFs (Selenium + Requests fallback)
# -----------------------------------------
def iniciar_sesion(driver):
    driver.get("https://myeforce.ecom.com.co/ecomltda/ingresar.php")
    time.sleep(2)
    driver.find_element(By.ID,"cta").send_keys(usuario)
    driver.find_element(By.ID,"ingr").send_keys(contraseña + Keys.RETURN)
    time.sleep(5)
    print("✅ Sesión iniciada correctamente.")

def obtener_sesion_requests(driver):
    s = requests.Session()
    for c in driver.get_cookies():
        s.cookies.set(c['name'], c['value'], domain=c.get('domain'))
    return s

def descargar_pdf_por_request(driver, url, destino):
    sess = obtener_sesion_requests(driver)
    resp = sess.get(url, stream=True)
    if resp.status_code==200:
        with open(destino,"wb") as f:
            for chunk in resp.iter_content(1024):
                f.write(chunk)
        print(f"✅ PDF guardado por request: {os.path.basename(destino)}")
    else:
        print(f"❌ error {resp.status_code} en request: {url}")

def esperar_por_patron(directorio, patrones, timeout=30):
    start=time.time()
    while time.time()-start<timeout:
        for f in glob.glob(os.path.join(directorio,"*")):
            if any(p in os.path.basename(f) for p in patrones):
                return f
        time.sleep(0.5)
    return None

def descargar_pdf_selenium(driver, url, patrones):
    driver.get(url); time.sleep(3)
    try:
        btn = WebDriverWait(driver,15).until(
            EC.element_to_be_clickable((By.XPATH,"//button[@id='download']"))
        )
        btn.click()
        print("⏳ intentando Selenium PDF...")
        found = esperar_por_patron(pdfs_path, patrones, timeout=30)
        if found:
            print(f"✅ descargado por Selenium: {os.path.basename(found)}")
            return found
    except Exception as e:
        print(f"⚠️ Selenium PDF falló: {e}")
    return None

def descargar_pdfs(driver):
    urls = {
        "doc":     f"https://myeforce.ecom.com.co//eforce/modulos/separacion/index.php?modo=imprimir&pla_numero={planilla_id}",
        "doc (1)": f"https://myeforce.ecom.com.co//eforce/modulos/planillasabono2/index.php?modo=imprimir&pla_numero={planilla_id}"
    }
    resultados = {}
    for key,out in [("doc","file1.pdf"),("doc (1)","file2.pdf")]:
        url = urls[key]
        path = descargar_pdf_selenium(driver, url, [key])
        if not path:
            fallback = os.path.join(pdfs_path, f"{key}.pdf")
            descargar_pdf_por_request(driver, url, fallback)
            path = fallback if os.path.exists(fallback) else None
        resultados[key] = path
    return resultados

# -----------------------------------------
# 9) GENERAR PAYLOAD FINAL
# -----------------------------------------
def generar_payload(pdf_paths):
    # PDFs
    pdf_entries = []
    for key,outname in [("doc","file1.pdf"),("doc (1)","file2.pdf")]:
        path = pdf_paths.get(key)
        if path and os.path.exists(path):
            raw = base64.b64encode(open(path,"rb").read()).decode()
            m = re.search(r"(JVBERi0x[\s\S]*)", raw)
            clean = m.group(1) if m else raw
            bstr = f"data:@file/pdf;base64,{clean}"
        else:
            bstr = ""
        pdf_entries.append({"base64":bstr,"fileType":"application/pdf","fileName":outname})

    # CSV ventas y datos: eliminar BOM antes de codificar
    def encode_csv(path):
        data = open(path,"rb").read()
        if data.startswith(b'\xef\xbb\xbf'):
            data = data[3:]
        return base64.b64encode(data).decode()

    sales = sorted(f for f in glob.glob(conversion_path+"/*.csv") if "SAsesor" in f)
    data  = sorted(f for f in glob.glob(conversion_path+"/*.csv") if "PAsesor" in f)
    csvSales = [{
        "base64": f"data:@file/csv;base64,{encode_csv(f)}",
        "fileType":"text/csv","fileName":f"sales{i}.csv"
    } for i,f in enumerate(sales,1)]
    csvData  = [{
        "base64": f"data:@file/csv;base64,{encode_csv(f)}",
        "fileType":"text/csv","fileName":f"data{i}.csv"
    } for i,f in enumerate(data,1)]

    addf = os.path.join(conversion_path, f"Entregas_{planilla_id}.csv")
    if os.path.exists(addf):
        addb = encode_csv(addf)
        add = {"fileName":"additional.csv","base64":f"data:@file/csv;base64,{addb}","fileType":"text/csv"}
    else:
        add = {"fileName":"additional.csv","base64":"","fileType":"text/csv"}

    return {
        "pdfFiles": pdf_entries,
        "csvFilesSales": csvSales,
        "csvFiles": csvData,
        "additionalCsvFile": add,
        "idDriver": idDriver
    }

def imprimir_payload_truncado(payload, lim=30):
    p = json.loads(json.dumps(payload))
    for k in ["pdfFiles","csvFilesSales","csvFiles"]:
        for it in p[k]:
            s = it["base64"]
            it["base64"] = s[:lim]+"..." if len(s)>lim else s
    s = p["additionalCsvFile"]["base64"]
    p["additionalCsvFile"]["base64"] = s[:lim]+"..." if len(s)>lim else s
    print(json.dumps(p, indent=4))

def guardar_payload_txt(payload):
    txt = os.path.join(download_path, f"payload_{planilla_id}_{idDriver}.txt")
    with open(txt,"w") as f:
        json.dump(payload, f, indent=2)
    print(f"💾 Payload guardado en {txt}")


def limpiar_archivos():
    # Borra todos los .xls
    for f in glob.glob(os.path.join(download_path, "*.xls")):
        os.remove(f)
    # Borra todos los .csv
    for f in glob.glob(os.path.join(conversion_path, "*.csv")):
        os.remove(f)
    # Borra todos los .pdf
    for f in glob.glob(os.path.join(pdfs_path, "*.pdf")):
        os.remove(f)
    print("🗑️ Archivos temporales eliminados (XLS, CSV, PDFs).")

def enviar_api(payload):
    url = "https://servimaxinternal.app/etlnodeapp/driverApp/extractBillsCopilatesFromCsv"
    headers = {"sede":"key","bogota":"value","Content-Type":"application/json"}
    r = requests.post(url, headers=headers, json=payload)
    if r.status_code == 200:
        print("🎉 Éxito! API devolvió 200")
        limpiar_archivos()
    else:
        print(f"❌ API error {r.status_code}: {r.text}")

# -----------------------------------------
# 10) WORKFLOW PRINCIPAL
# -----------------------------------------
driver = webdriver.Chrome(options=chrome_options)
try:
    # LOGIN
    iniciar_sesion(driver)

    # XLS
    descargar_pedidos(driver, asesor1, "PAsesorUno")
    descargar_facturas(driver, asesor1, "SAsesorUno")
    descargar_pedidos(driver, asesor2, "PAsesorDos")
    descargar_facturas(driver, asesor2, "SAsesorDos")
    descargar_entregas(driver)

    # CSV
    convertir_a_csv_convertapi()

    # PDF
    pdf_paths = descargar_pdfs(driver)

    # PAYLOAD
    payload = generar_payload(pdf_paths)
    print("\n🔔 Payload truncado:")
    imprimir_payload_truncado(payload)

    # GUARDAR Y ENVIAR
    guardar_payload_txt(payload)
    enviar_api(payload)

finally:
    driver.quit()
